# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)

import os
from sqlalchemy import create_engine
import re
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

import warnings

warnings.simplefilter('ignore')

In [2]:
# import libraries
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/mafda/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/mafda/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/mafda/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
# load data from database
database_filepath = "../data/DisasterResponse.db"
engine = create_engine('sqlite:///' + database_filepath)
table_name = os.path.basename(database_filepath).replace(".db","") + "_table"
df = pd.read_sql_table(table_name, engine)
df.shape

(26216, 39)

In [4]:
df.head(2)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0


In [5]:
# Extract X and y variables from the data for the modelling
X = df['message']
y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

print(f'X shape {X.shape}, y shape {y.shape}')

X shape (26216,), y shape (26216, 35)


### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    words = nltk.word_tokenize(text)
    
    # remove stop words
    stopwords_ = nltk.corpus.stopwords.words("english")
    words = [word for word in words if word not in stopwords_]
    
    # extract root form of words
    words = [nltk.stem.WordNetLemmatizer().lemmatize(word, pos='v') for word in words]

    return words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline_fitted = pipeline.fit(X_train, y_train)

In [9]:
random_message = ["I am hungry, I don't have food to eat, I don't have house I don't have clothes I count on you thank you so much"]
test_output = pipeline.predict(random_message)
print(y_train.columns.values[(test_output.flatten()==1)])

['related' 'request' 'aid_related' 'food' 'direct_report']


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
def get_evaluate_multioutput(actual, predicted, col_names):
    """Calculate evaluation metrics for ML model
    
    Args: actual: array. Array containing actual labels. predicted: array. Array
    containing predicted labels. col_names: list of strings. List containing
    names for each of the predicted fields.
       
    Returns: metrics_df: dataframe. Dataframe containing the accuracy,
    precision, recall and f1 score for a given set of actual and predicted
    labels.
    """
    metrics = []
    
    # Calculate evaluation metrics for each set of labels
    for i in range(len(col_names)):
        accuracy = accuracy_score(actual[:, i], predicted[:, i])
        precision = precision_score(actual[:, i], predicted[:, i], average='weighted')
        recall = recall_score(actual[:, i], predicted[:, i], average='weighted')
        f1 = f1_score(actual[:, i], predicted[:, i], average='weighted')
        
        metrics.append([accuracy, precision, recall, f1])
    
    # Create dataframe containing metrics
    metrics = np.array(metrics)
    metrics_df = pd.DataFrame(data = metrics, index = col_names, columns = ['Accuracy', 'Precision', 'Recall', 'F1'])
      
    return metrics_df    

In [11]:
# Calculate evaluation metrics for test set
y_pred = pipeline.predict(X_test)
col_names = list(y.columns.values)

In [12]:
print(get_evaluate_multioutput(np.array(y_test), y_pred, col_names))

                        Accuracy  Precision    Recall        F1
related                 0.818126   0.804447  0.818126  0.801469
request                 0.895484   0.891561  0.895484  0.883636
offer                   0.995117   0.990259  0.995117  0.992682
aid_related             0.775404   0.774046  0.775404  0.774372
medical_help            0.920201   0.898596  0.920201  0.890287
medical_products        0.956057   0.948991  0.956057  0.938823
search_and_rescue       0.973299   0.970262  0.973299  0.962003
security                0.980928   0.963703  0.980928  0.972239
military                0.967653   0.960519  0.967653  0.954295
water                   0.957125   0.954925  0.957125  0.948203
food                    0.940494   0.936484  0.940494  0.935066
shelter                 0.936070   0.929631  0.936070  0.923925
clothing                0.985658   0.984297  0.985658  0.979711
money                   0.976961   0.977492  0.976961  0.966553
missing_people          0.988709   0.981

### 6. Improve your model
Use grid search to find better parameters. 

In [13]:
from sklearn.model_selection import GridSearchCV

In [14]:
parameters = {'vect__min_df': [1, 5],
            #   'tfidf__use_idf':[True, False],
            #   'clf__estimator__n_estimators':[10, 25], 
              'clf__estimator__min_samples_split':[2, 5, 10]}


cv = GridSearchCV(pipeline, param_grid=parameters, scoring='f1_micro', verbose = 10)
cv.fit(X_train, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5; 1/6] START clf__estimator__min_samples_split=2, vect__min_df=1.........
[CV 1/5; 1/6] END clf__estimator__min_samples_split=2, vect__min_df=1;, score=nan total time= 2.8min
[CV 2/5; 1/6] START clf__estimator__min_samples_split=2, vect__min_df=1.........
[CV 2/5; 1/6] END clf__estimator__min_samples_split=2, vect__min_df=1;, score=nan total time= 2.9min
[CV 3/5; 1/6] START clf__estimator__min_samples_split=2, vect__min_df=1.........
[CV 3/5; 1/6] END clf__estimator__min_samples_split=2, vect__min_df=1;, score=nan total time= 2.9min
[CV 4/5; 1/6] START clf__estimator__min_samples_split=2, vect__min_df=1.........
[CV 4/5; 1/6] END clf__estimator__min_samples_split=2, vect__min_df=1;, score=nan total time= 2.9min
[CV 5/5; 1/6] START clf__estimator__min_samples_split=2, vect__min_df=1.........
[CV 5/5; 1/6] END clf__estimator__min_samples_split=2, vect__min_df=1;, score=nan total time= 2.9min
[CV 1/5; 2/6] START clf__estim

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x15c135f80>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__min_samples_split': [2, 5, 10],
                         'vect__min_df': [1, 5]},
             scoring='f1_micro', verbose=10)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.